# CNN, First model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
import pickle
import numpy as np

In [13]:
# import data
with open('X_train_even.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
    
with open('Y_train_even.pickle', 'rb') as handle:
    Y_train = pickle.load(handle)
    
with open('X_test_even.pickle', 'rb') as handle:
    X_test = pickle.load(handle)
    
with open('Y_test_even.pickle', 'rb') as handle:
    Y_test = pickle.load(handle)
    
# small change in format
tmp = np.concatenate(X_train).ravel()
X_train = np.reshape(tmp,(len(X_train),100))

tmp = np.concatenate(X_test).ravel()
X_test = np.reshape(tmp,(len(X_test),100))
review_length = X_test.shape[1]

print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
print('The length of each review is {}.'.format(review_length))


Loaded dataset with 199483 training samples, 49871 test samples
The length of each review is 100.


In [18]:
num_datapoints = 10000

small_Y = Y_train[0:num_datapoints]
small_X = X_train[0:num_datapoints]

The share of negative reviews:
0.4924
It is approximately even!


In [23]:
model = Sequential()
e = Embedding(100000, 200, input_length=review_length)
model.add(e)
model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#model.fit(X_train, Y_train, validation_split=0.2, epochs=3, batch_size=32, verbose=2)

model.fit(small_X, small_Y, validation_split=0.2, epochs=10, batch_size=32, verbose=1)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 76s 10ms/step - loss: 0.5016 - acc: 0.7488 - val_loss: 0.4071 - val_acc: 0.8160
Epoch 2/10
8000/8000 [==============================] - 73s 9ms/step - loss: 0.2309 - acc: 0.9116 - val_loss: 0.4144 - val_acc: 0.8255
Epoch 3/10
8000/8000 [==============================] - 75s 9ms/step - loss: 0.0551 - acc: 0.9856 - val_loss: 0.5120 - val_acc: 0.8225
Epoch 4/10
4032/8000 [==============>...............] - ETA: 36s - loss: 0.0107 - acc: 0.9993

KeyboardInterrupt: 